In [9]:
import sys
import json
import pathlib

import joblib
import pandas as pd
from pycytominer.cyto_utils.features import infer_cp_features

# project module imports
sys.path.append("../../")  # noqa
from src.utils import check_feature_order  # noqa

## Setting up file paths and parameters 

In [10]:
# setting up paths
results_dir = pathlib.Path("../../results")
data_split_dir = (results_dir / "1.data_splits/").resolve(strict=True)
jump_data_dir = pathlib.Path("../../data/JUMP_data").resolve(strict=True)
modeling_dir = pathlib.Path("../../results/2.modeling").resolve(strict=True)


# data files
jump_data_path = (jump_data_dir / "JUMP_all_plates_normalized_negcon.csv.gz").resolve(
    strict=True
)
multi_class_model_path = (modeling_dir / "multi_class_model.joblib").resolve(
    strict=True
)
feature_col_names = (data_split_dir / "feature_cols.json").resolve(strict=True)


# output paths
jump_analysis_dir = (results_dir / "3.jump_analysis").resolve()
jump_analysis_dir.mkdir(exist_ok=True)

## loading files

In [11]:
# loading in the negatlive controled normalized profiles
jump_df = pd.read_csv(jump_data_path)

# loading json file containing selected feature names
with open(feature_col_names, mode="r") as infile:
    cell_injury_cp_feature_cols = json.load(infile)

# loading json file that contains the coder and decoder injury labels
with open(data_split_dir / "injury_codes.json") as infile:
    injury_codes = json.load(infile)

# display dataframe and size
print("JUMP dataset size:", jump_df.shape)
jump_df.head()

JUMP dataset size: (19498, 5805)


,Metadata_broad_sample,Metadata_Plate,Metadata_Well,Metadata_gene,Metadata_pert_type,Metadata_control_type,Metadata_target_sequence,Metadata_negcon_control_type,Cells_AreaShape_Area,Cells_AreaShape_BoundingBoxArea,...,Nuclei_Texture_Variance_RNA_3_03_256,Nuclei_Texture_Variance_RNA_5_00_256,Nuclei_Texture_Variance_RNA_5_01_256,Nuclei_Texture_Variance_RNA_5_02_256,Nuclei_Texture_Variance_RNA_5_03_256,Metadata_solvent,Metadata_InChIKey,Metadata_pert_iname,Metadata_pubchem_cid,Metadata_smiles
0,BRDN0001480888,BR00116997,A01,HIF1A,trt,NaN,TATGTGTGAATTACGTTGTG,NaN,1.365900,2.159800,...,-1.448300,-1.386300,-1.368800,-1.426400,-1.323900,NaN,NaN,NaN,NaN,NaN
1,BRDN0001483495,BR00116997,A02,CATSPER4,trt,NaN,CCGACCGTAGGACTCGTGAA,NaN,-0.577230,-0.328600,...,-0.787660,-0.782950,-0.757350,-0.770820,-0.758180,NaN,NaN,NaN,NaN,NaN
2,BRDN0001147364,BR00116997,A03,DDR2,trt,NaN,CCGTGACAAACCGAGCACTG,NaN,-0.348260,-0.077757,...,-0.612140,-0.606210,-0.582380,-0.604900,-0.569400,NaN,NaN,NaN,NaN,NaN
3,BRDN0001490272,BR00116997,A04,OPRL1,trt,NaN,AACGGGAACACCGACAACAG,NaN,-0.076636,0.302890,...,0.001638,0.005891,0.029338,-0.014926,0.055581,NaN,NaN,NaN,NaN,NaN
4,BRDN0001480510,BR00116997,A05,SLC7A11,trt,NaN,GAAGAGATTCAAGTATTACG,NaN,-0.282870,-0.064418,...,-0.431160,-0.401550,-0.374840,-0.423030,-0.343840,NaN,NaN,NaN,NaN,NaN


## Feature alignment
In this section, we are identifying the shared features present in both the cell injury and JUMP datasets. 
Once these features are identified, we update the JUMP dataset to include only those features that are shared between both profiles for our machine learning application.

First we identify the CellProfiler (CP) features present in the JUMP data. 
We accomplish this by utilizing `pycytominer`'s  'infer_cp_features()', which helps us identify CP features in the JUMP dataset.

In [12]:
# get compartments
metadata_prefix = "Metadata_"
compartments = list(
    set(
        [
            feature_name.split("_")[0]
            for feature_name in jump_df.columns.tolist()
            if not feature_name.startswith("Metadata_")
        ]
    )
)

# find CP features in JUMP dataset
jump_cp_features = infer_cp_features(jump_df, compartments=compartments)
meta_features = infer_cp_features(jump_df, compartments=compartments, metadata=True)

# display number of features of both profiles
print("Number of Metadata Features:", len(meta_features))
print(
    "Number of CP features that cell injury has",
    len(cell_injury_cp_feature_cols["feature_cols"]),
)
print("Number of CP features that JUMP has:", len(jump_cp_features))

Number of Metadata Features: 13
Number of CP features that cell injury has 346
Number of CP features that JUMP has: 5792


Now that we have identified the features present in both datasets, the next step is to align them. This involves identifying the common features between both profiles and utilizing these features to update our JUMP dataset for our machine learning model.

In [13]:
cell_injury_cp_features = cell_injury_cp_feature_cols["feature_cols"]

# finding shared featues using intersection
aligned_features = list(set(cell_injury_cp_features) & set(jump_cp_features))

# displaying number of shared features between both profiles
print("Number of shapred features of both profiles", len(aligned_features))

Number of shapred features of both profiles 207


The objective of this step is to preserve the order of the feature space.

Since we have identified the shared feature space across both profiles, we still need to address those that are missing. 
Therefore, to maintain the feature space order, we opted to use the cell injury feature space as our reference feature space order, as our multi-class model was trained to understand this specific order.

Next, we addressed features that were not found within the JUMP dataset.
This was done by including them in the alignment process, but defaulted their values to 0.

Ultimately, we generated a new profile called `aligned_jump_df`, which contains the correctly aligned and ordered feature space from the cell injury dataset.

In [14]:
# multiplier is the number of samples in JUMP data
# this is used to default non-aligned features to 0
multiplier = jump_df.shape[0]

# storing feature and values in order
aligned_jump = {}
for injury_feat in cell_injury_cp_features:
    if injury_feat not in aligned_features:
        aligned_jump[injury_feat] = [0.0] * multiplier
    else:
        aligned_jump[injury_feat] = jump_df[injury_feat].values.tolist()

# creating dataframe with the aligned features and retained feature order
aligned_jump_feats_df = pd.DataFrame.from_dict(aligned_jump, orient="columns")

# sanity check: see if the feature order in the `cell_injury_cp_feature_cols` is the same with
# the newly generated aligned JUMP dataset
assert (
    cell_injury_cp_features == aligned_jump_feats_df.columns.tolist()
), "feature space are not aligned"
assert check_feature_order(
    ref_feat_order=cell_injury_cp_features,
    input_feat_order=aligned_jump_feats_df.columns.tolist(),
), "feature space do not follow the same order"

## Applying to our Multi-Class trained model

We applying the aligned JUMP dataset to our trained multi-class model and measure the probabiltiies of which cell injury each well possessed. 

In [15]:
# loading in mutliclass model
multi_class_cell_injury_model = joblib.load(multi_class_model_path)

In [16]:
# apply
pred_proba = multi_class_cell_injury_model.predict_proba(aligned_jump_feats_df)

# convert prediction probabilities to a pandas daraframe
pred_proba_df = pd.DataFrame(pred_proba)

# update the column names with the name of the injury class
pred_proba_df.columns = [
    injury_codes["decoder"][str(colname)] for colname in pred_proba_df.columns.tolist()
]

# augment the porbability scores by meringing with the metadata associated with each well
pred_proba_df = jump_df[meta_features].merge(
    pred_proba_df, left_index=True, right_index=True
)

# # save the probability dataframe
pred_proba_df.to_csv(jump_analysis_dir / "JUMP_injury_proba.csv.gz", compression="gzip")

# # display shape and size
print("Probability shape:", pred_proba_df.shape)
pred_proba_df.head()

Probability shape: (19498, 28)


,Metadata_broad_sample,Metadata_Plate,Metadata_Well,Metadata_gene,Metadata_pert_type,Metadata_control_type,Metadata_target_sequence,Metadata_negcon_control_type,Metadata_solvent,Metadata_InChIKey,...,Miscellaneous,Redox,HDAC,mTOR,Proteasome,Saponin,Mitochondria,Ferroptosis,Tannin,Nonspecific reactive
0,BRDN0001480888,BR00116997,A01,HIF1A,trt,NaN,TATGTGTGAATTACGTTGTG,NaN,NaN,NaN,...,1.517926e-17,1.411549e-12,1.000000,9.999998e-01,4.057728e-10,6.991383e-20,7.475609e-13,1.000000,6.891960e-51,1.0
1,BRDN0001483495,BR00116997,A02,CATSPER4,trt,NaN,CCGACCGTAGGACTCGTGAA,NaN,NaN,NaN,...,4.015459e-19,1.000000e+00,1.000000,7.511064e-03,1.000000e+00,1.474707e-23,5.653147e-02,1.000000,3.635383e-50,1.0
2,BRDN0001147364,BR00116997,A03,DDR2,trt,NaN,CCGTGACAAACCGAGCACTG,NaN,NaN,NaN,...,1.614496e-19,9.987153e-01,1.000000,2.588650e-09,1.000000e+00,1.871569e-05,2.378226e-06,1.000000,1.825883e-50,1.0
3,BRDN0001490272,BR00116997,A04,OPRL1,trt,NaN,AACGGGAACACCGACAACAG,NaN,NaN,NaN,...,1.667038e-10,1.000000e+00,0.999995,5.331151e-05,9.999999e-01,3.876785e-17,1.000000e+00,1.000000,4.502330e-57,1.0
4,BRDN0001480510,BR00116997,A05,SLC7A11,trt,NaN,GAAGAGATTCAAGTATTACG,NaN,NaN,NaN,...,1.605561e-13,1.000000e+00,1.000000,7.591227e-01,1.000000e+00,1.202547e-16,9.818902e-01,0.999619,1.270453e-67,1.0
